In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

from pyphocorehelpers.print_helpers import CapturedException
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers

# BATCH_DATE_TO_USE = '2023-10-20' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2023-10-18_Apogee' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2023-10-27_Lab' # used for filenames throught the notebook

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')] # , Path(r'/home/halechr/FastData')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    # display(good_only_batch_progress_df)
    display(batch_progress_df)

Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_2023-10-27_Lab.pkl... done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-08 14:26:15,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-09 01:22:43,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,2006-06-09 03:23:37,0,0,0,0,False,,,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-12 15:55:31,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,2009-11-04 21:20:03,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,False,True
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/KDIBA/pin01/one/redundant,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/KDIBA/pin01/one/showclus,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/KDIBA/pin01/one/sleep,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False


# Run Batch Executions/Computations

In [3]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)
good_session_concrete_folders


from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import generate_batch_single_session_scripts

## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
included_session_contexts, output_python_scripts, output_slurm_scripts = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts, output_directory=Path('output/generated_slurm_scripts/').resolve(), use_separate_run_directories=True)
output_python_scripts

included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    display(included_session_batch_progress_df)

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-08 14:26:15,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-09 01:22:43,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-12 15:55:31,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-07 16:40:19,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,True,True
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-06-08 21:16:25,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-1...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,kdiba,vvp01,two,2006-4-10_12-58-3,kdiba_vvp01_two_2006-4-10_12-58-3,/media/MAX/Data/KDIBA/vvp01/two/2006-4-10_12-58-3,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2006-04-10 12:58:03,0,0,0,0,False,/media/MAX/Data/KDIBA/vvp01/two/2006-4-10_12-5...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-10_12-5...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-10_12-5...,True,True
52,kdiba,pin01,one,11-02_17-46-44,kdiba_pin01_one_11-02_17-46-44,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2009-11-02 17:46:44,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,True,True
53,kdiba,pin01,one,11-02_19-28-0,kdiba_pin01_one_11-02_19-28-0,/media/MAX/Data/KDIBA/pin01/one/11-02_19-28-0,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2009-11-02 19:28:00,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/11-02_19-28-0/...,/media/MAX/Data/KDIBA/pin01/one/11-02_19-28-0/...,/media/MAX/Data/KDIBA/pin01/one/11-02_19-28-0/...,True,True
54,kdiba,pin01,one,11-03_12-3-25,kdiba_pin01_one_11-03_12-3-25,/media/MAX/Data/KDIBA/pin01/one/11-03_12-3-25,SessionBatchProgress.FAILED,!! Must have 'label' column. ::::: (<class 'At...,2009-11-03 12:03:25,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/11-03_12-3-25/...,/media/MAX/Data/KDIBA/pin01/one/11-03_12-3-25/...,/media/MAX/Data/KDIBA/pin01/one/11-03_12-3-25/...,True,True


# Execute Batch

In [4]:
# %pdb on

# multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=3)
  
# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
# included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()

# Only require sessions to have replay annotations:
# included_session_contexts = batch_progress_df[batch_progress_df['has_user_replay_annotations']]['context'].to_numpy().tolist()

# included_session_contexts = batch_progress_df['context'].to_numpy().tolist()[:4] # Only get the first 6
# Limit the contexts to run to the last N:
# included_session_contexts = included_session_contexts[3:5]

# included_session_contexts = [included_session_contexts[3]]

# ALL
included_session_contexts = included_session_contexts

# ## No filtering the sessions:
# included_session_contexts = None

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts

# # No recomputing at all:
# result_handler = BatchSessionCompletionHandler(force_reload_all=False,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=SavingOptions.NEVER, override_file=),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=SavingOptions.NEVER),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
#                                                 **multiprocessing_kwargs)

# # No Reloading
# result_handler = BatchSessionCompletionHandler(force_reload_all=False,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
#                                                 **multiprocessing_kwargs)


# # Forced Reloading:
result_handler = BatchSessionCompletionHandler(force_reload_all=True,
                                                session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
                                                global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
                                                **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session


# def a_test_completion_function(self, global_data_root_parent_path, curr_session_context, curr_session_basedir, curr_active_pipeline, across_session_results_extended_dict: dict) -> dict:
#     # print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#     print(f'<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
#     print(f'a_test_completion_function(curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}, ...,across_session_results_extended_dict: {across_session_results_extended_dict})')
#     long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
#     # long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
#     # long_results, short_results, global_results = [curr_active_pip eline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
#     # Get existing laps from session:
#     # long_laps, short_laps, global_laps = [curr_active_pipeline.filtered_sessions[an_epoch_name].laps.as_epoch_obj() for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
#     # long_replays, short_replays, global_replays = [Epoch(curr_active_pipeline.filtered_sessions[an_epoch_name].replay.epochs.get_valid_df()) for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
#     # long_PBEs, short_PBEs, global_PBEs = [curr_active_pipeline.filtered_sessions[an_epoch_name].pbe for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]


#     output_file_prefix = curr_session_context.get_description(separator="|", include_property_names=False)
#     print(f'-----------------_____------______---- BEGIN Directional Laps Result: \n\toutput_file_prefix: {output_file_prefix}\n\tcurr_active_pipeline.active_config_names: {curr_active_pipeline.active_config_names}')
#     directional_laps_result = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']['computed_base_epoch_names']
#     print(f"\tsplit_directional_laps_names: {curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']['split_directional_laps_names']}")
#     print(f"\tsplit_directional_laps_names: {curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']['computed_base_epoch_names']}")
#     print(f'\n__ End Directional Laps result')

#     # jonathan_firing_rate_analysis_result = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
#     # (epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
#     # neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.05)

#     # ## Output the BatchPhoJonathanFiguresHelper
#     # fig_1c_figures_all_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df.sort_values('custom_frs_index', ascending=True, inplace=False), included_unit_neuron_IDs=None,
# 	# n_max_page_rows=20, write_vector_format=False, write_png=True,
# 	# show_only_refined_cells=False, disable_top_row=False, split_by_short_long_shared=False)

    
#     # global_replays.filename = Path(f"output/{output_file_prefix}_global_replays").resolve()
#     # print(f'global_replays.filename: {global_replays.filename}')
#     # global_replays.to_neuroscope()

#     # global_PBEs.filename = Path(f"output/{output_file_prefix}_global_PBEs").resolve()
#     # print(f'global_PBEs.filename: {global_PBEs.filename}')
#     # global_PBEs.to_neuroscope('PBE')


#     # curr_active_pipeline, directional_lap_specific_configs = DirectionalLapsHelpers.split_to_directional_laps(curr_active_pipeline, add_created_configs_to_pipeline=True)
#     # curr_active_pipeline, directional_lap_specific_configs = constrain_to_laps(curr_active_pipeline)
#     # list(directional_lap_specific_configs.keys())

#     # joined_neruon_fri_df = build_merged_neuron_firing_rate_indicies(curr_active_pipeline, enable_display_intermediate_results=False)
#     # AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df')
#     print(f'>>\t done with {output_file_prefix}')
#     print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#     print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')

#     return across_session_results_extended_dict


# result_handler.completion_functions.append(a_test_completion_function)

## Specific Setup for 2023-09-28 Changes to LxC/SxC "refinements"
result_handler.extended_computations_include_includelist = ['pf_computation', 'pfdt_computation', 'firing_rate_trends',
                                                # 'pf_dt_sequential_surprise',
                                                # 'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                'position_decoding_two_step', 
                                                'long_short_decoding_analyses',
                                                'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                                'long_short_inst_spike_rate_groups',
                                                'long_short_endcap_analysis',
                                                # 'split_to_directional_laps'
                                                ]


basic_local_computations = ['pf_computation', 'pfdt_computation', 'firing_rate_trends',
#                                                 'pf_dt_sequential_surprise',
                                                # 'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                #'position_decoding_two_step', 
                                                ]
 
# result_handler.extended_computations_include_includelist = ['long_short_inst_spike_rate_groups']


result_handler.enable_hdf5_output = True # output the HDF5 when done.
# result_handler.override_existing_frs_index_values = True
# result_handler.frs_index_inclusion_magnitude = 0.1

# result_handler.enable_hdf5_output = False
result_handler.override_existing_frs_index_values = False


## Execute with the custom arguments.
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=False, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': basic_local_computations,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

len(included_session_contexts): 15
Beginning processing with len(included_session_contexts): 15
build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-08_14-26-15"):build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-12_15-55-31"):build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-09_1-22-43"):


	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-12_15-55-31 has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-12_15-55-31.log	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-08_14-26-15 has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-08_14-26-15.log
	 Batch Ta

/home/halechr/repos/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:123: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/2006-6-09_1-22-43.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


Skipping loading from pickled file because force_reload == True.Skipping loading from pickled file because force_reload == True.

Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/2006-6-09_1-22-43.epochs_info.mat... 

/home/halechr/repos/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:123: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/2006-6-12_15-55-31.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')
/home/halechr/repos/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:123: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/2006-6-08_14-26-15.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/2006-6-12_15-55-31.epochs_info.mat...Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/2006-6-08_14-26-15.epochs_info.mat...  done.
done.done.

Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/2006-6-09_1-22-43.position_info.mat... Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/2006-6-08_14-26-15.position_info.mat...Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/2006-6-12_15-55-31.position_info.mat...  done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/2006-6-12_15-55-31.spikes.mat...done. 
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/2006-6-09_1-22-43.spikes.mat... done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20

...

two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (89250,)
Performing run_specific_computations_single_context on filtered_session with filter named "maze_any_even_laps"...


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/Analysis/Decoder/reconstruction.py:367: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (90848,)
	 done.
Exception occured while computing (`perform_specific_computation(...)`) or validating (`validate_computation_test(...)`) after recomputation:
 Inner exception: 'pf1D_TwoStepDecoder'
ERROR perform `batch_extended_computations` or saving GLOBAL COMPUTATION RESULTS for pipeline of curr_session_context: kdiba_pin01_one_fet11-01_12-58-54. error: !! 'pf1D_TwoStepDecoder' ::::: (<class 'KeyError'>, KeyError('pf1D_TwoStepDecoder'), <traceback object at 0x7f827de6f880>)
no changes in global results.
Saving mode == ALWAYS so trying to save despite no changes.
global_computation_results_pickle_path: /media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl
Saving (file mode '/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl') saved session pickle file results : /media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computa

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'fet11-01_12-58-54'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


ERROR: encountered exception !! 'long_short_fr_indicies_analysis' ::::: (<class 'KeyError'>, KeyError('long_short_fr_indicies_analysis'), <traceback object at 0x7f8266089c00>) while trying to build the session HDF output.
ERROR: encountered exception !! 'long_short_fr_indicies_analysis' ::::: (<class 'KeyError'>, KeyError('long_short_fr_indicies_analysis'), <traceback object at 0x7f826617a1c0>) while trying to build the session HDF output for kdiba_pin01_one_fet11-01_12-58-54
	 doing specific instantaneous firing rate computation for context: kdiba_pin01_one_fet11-01_12-58-54...
WARN: on_complete_success_execution_session: encountered exception !! 'jonathan_firing_rate_analysis' ::::: (<class 'KeyError'>, KeyError('jonathan_firing_rate_analysis'), <traceback object at 0x7f8265d07ac0>) while trying to compute the instantaneous firing rates and set self.across_sessions_instantaneous_fr_dict[kdiba_pin01_one_fet11-01_12-58-54]
"========================== END BATCH =========================

In [5]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
global_batch_run.to_hdf(file_path,'/')

Saving (file mode '/media/MAX/Data/global_batch_result_2023-10-27_Lab.pkl') saved session pickle file results : /media/MAX/Data/global_batch_result_2023-10-27_Lab.pkl... 	moving new output at '/media/MAX/Data/20231027231538-global_batch_result_2023-10-27_Lab.pkltmp' -> to desired location: '/media/MAX/Data/global_batch_result_2023-10-27_Lab.pkl'
done.


/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-08_14-26-15'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-12_15-55-31'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-07_16-40-19'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natu

done outputting HDF file.


/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'fet11-01_12-58-54'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [6]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,2006-06-08 14:26:15,40,354,40,272,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,2006-06-12 15:55:31,40,57,33,77,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,2006-06-07 16:40:19,41,287,39,447,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,True,True
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,SessionBatchProgress.COMPLETED,None,2006-06-09 22:24:40,47,155,40,568,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-2...,True,True
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,SessionBatchProgress.COMPLETED,None,2006-06-12 16:53:46,40,69,40,58,True,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,True,True
31,kdiba,vvp01,two,2006-4-09_16-40-54,kdiba_vvp01_two_2006-4-09_16-40-54,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,SessionBatchProgress.COMPLETED,None,2006-04-09 16:40:54,48,44,49,26,True,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-4...,True,True
52,kdiba,pin01,one,11-02_17-46-44,kdiba_pin01_one_11-02_17-46-44,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44,SessionBatchProgress.COMPLETED,None,2009-11-02 17:46:44,41,138,40,156,True,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44...,True,True
64,kdiba,pin01,one,fet11-01_12-58-54,kdiba_pin01_one_fet11-01_12-58-54,/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54,SessionBatchProgress.COMPLETED,None,2009-11-01 12:58:54,60,601,25,276,True,/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58...,/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58...,/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58...,True,True


# Across Sessions After Batching Complete

In [ ]:
a_batch_progress_df = included_session_batch_progress_df.copy()

good_session_concrete_folders = [ConcreteSessionFolder(a_context, a_basedir) for a_context, a_basedir in zip(list(a_batch_progress_df.context.values), list(a_batch_progress_df.basedirs.values))]

# good_only_batch_progress_df.batch_results
# included_h5_paths = [get_file_str_if_file_exists(v.joinpath('output','pipeline_results.h5').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]
# included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]

In [ ]:
# target_dir = Path('output/across_session_results/2023-09-29').resolve()
# target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-09-29').resolve()
# target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-10-03').resolve()
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=target_dir)
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix='2023-10-05', only_include_file_types=['local_pkl', 'global_pkl','h5'])
copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['local_pkl', 'global_pkl'])
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix='2023-10-07', only_include_file_types=['local_pkl', 'global_pkl','h5'])
copy_dict

In [ ]:
moved_files_dict_h5_files = copy_movedict(copy_dict)
moved_files_dict_h5_files

In [ ]:
moved_files_copydict_output_filename=f'backed_up_files_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = Path(global_data_root_parent_path).joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: {moved_files_copydict_file_path}')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

In [ ]:
read_moved_files_dict_files = read_copydict_from_text_file(moved_files_copydict_file_path, debug_print=False)
read_moved_files_dict_files

In [ ]:
# read_moved_files_dict_files
restore_moved_files_dict_files = invert_filedict(read_moved_files_dict_files)
restore_moved_files_dict_files

In [ ]:
check_output_h5_files(included_h5_paths)

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

In [ ]:
# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation
across_sessions_recomputed_instantaneous_fr_dict = {}

# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
        a_good_recomp_result = a_result.across_session_results.get('recomputed_inst_fr_comps', None)
        if a_good_recomp_result is not None:
            across_sessions_recomputed_instantaneous_fr_dict[a_ctxt] = a_good_recomp_result
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
across_sessions_recomputed_instantaneous_fr_dict

In [ ]:
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')


AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
                                                 inst_fr_output_filename=f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')



# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables

# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'{BATCH_DATE_TO_USE}')

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)
# neuron_replay_stats_table['is_refined_LxC']

In [ ]:
long_short_fr_indicies_analysis_table

In [ ]:
neuron_replay_stats_table

In [ ]:
neuron_identities_table

In [ ]:
# np.sum(neuron_replay_stats_table['is_refined_LxC'])
# np.isnan(neuron_replay_stats_table['is_refined_LxC'])

In [ ]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
# _out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
# _out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership','is_refined_LxC', 'is_refined_SxC'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'is_refined_LxC', 'is_refined_SxC', 'count']
_out_value_counts_df

In [ ]:
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()

## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')

# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']

merged_df

In [ ]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
_out_value_counts_df

In [ ]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

In [ ]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
df.plot()


In [ ]:

from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Extract output files from all completed sessions:

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent

def save_filelist_to_text_file(hdf5_output_paths, filelist_path: Path):
    _out_string = '\n'.join([str(a_file) for a_file in hdf5_output_paths])
    print(f'{_out_string}')
    print(f'saving out to "{filelist_path}"...')
    with open(filelist_path, 'w') as f:
        f.write(_out_string)
    return _out_string, filelist_path

# Save output filelist:

# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}
session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)

# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from neuropy.core.epoch import Epoch

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
file_path
global_batch_run.to_hdf(file_path,'/')

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# good_session_batch_outputs = global_batch_run.session_batch_outputs

sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
[a_ctxt.get_initialization_code_string() for a_ctxt in sessions_with_results]

# OLD

# 2023-10-06 - `joined_neruon_fri_df` loading

In [ ]:
# BATCH_DATE_TO_USE = '2023-10-05_NewParameters'
BATCH_DATE_TO_USE = '2023-10-07'
all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE)


In [ ]:

joined_neruon_fri_df_basename = f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df'
AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=joined_neruon_fri_df_basename, include_csv=False)
print(f'>>\t done with {output_file_prefix}')

# 2023-10-04 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'2023-10-07') # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

In [ ]:
neuron_replay_stats_table

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

# for a_ctx, a_val in annotation_man.get_hardcoded_specific_session_override_dict().items():
# 	annotation_man.annotations[a_ctx] = a_val

# for a_ctx, a_val in UserAnnotationsManager.get_user_annotations().items():
# 	annotation_man.annotations[a_ctx] = a_val

# for a_ctx, a_val in session_cell_exclusivity_annotations.items():
# 	# Not ideal. Adds a key 'session_cell_exclusivity' to the extant session context instead of being indexable by an entirely new context
# 	annotation_man.annotations[a_ctx] = annotation_man.annotations.get(a_ctx, {}) | dict(session_cell_exclusivity=a_val)
# 	# annotation_man.annotations[a_ctx.overwriting_context(user_annotation='session_cell_exclusivity')] = a_val

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table

In [ ]:
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='x_frs_index', title='Pf Peak position vs. LapsFRI', ylabel='Lap FRI')

long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='y_frs_index', title='Pf Peak position vs. ReplayFRI', ylabel='Replay FRI')

 #TODO 2023-10-05 11:40: - [ ] Extract the "contrarian cells", the ones that have a strong exclusivity on the laps but the opposite tendency on the replays


In [ ]:
long_short_fr_indicies_analysis_table.to_csv('output/2023-10-07_long_short_fr_indicies_analysis_table.csv')

# 2023-10-10 - Statistics for `across_sessions_bar_graphs`, analysing `across_session_inst_fr_computation` 

In [ ]:
import scipy.stats as stats
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import pho_stats_perform_diagonal_line_binomial_test, pho_stats_bar_graph_t_tests

binom_test_chance_result = pho_stats_perform_diagonal_line_binomial_test(long_short_fr_indicies_analysis_table)
print(f'binom_test_chance_result: {binom_test_chance_result}')

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

## 2023-10-04 - Run `AcrossSessionsVisualizations` corresponding to the PhoDibaPaper2023 figures for all sessions


In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
ann_man = UserAnnotationsManager()
included_annotations = {ctxt:ann_man.annotations[ctxt].get('session_cell_exclusivity', None) for ctxt in included_session_contexts}

all_LxCs = []
all_SxCs = []

for ctxt, an_ann in included_annotations.items():
	session_ctxt_key:str = ctxt.get_description(separator='|', subset_includelist=IdentifyingContext._get_session_context_keys())
	all_LxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.LxC])
	all_SxCs.extend([f"{session_ctxt_key}|{aclu}" for aclu in an_ann.SxC])
	
all_LxCs

In [ ]:
all_SxCs

In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
_out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

In [ ]:
## 2023-10-11 - Surprise Shuffling
